***
## RT Beacons
***

In [ ]:
using ODBC
using DataFrames
using DSWB
using Formatting

dsn = "dswb-natgeo" # Redshift endpoint
table = "beacons_4744" # beacon table name
tableRt = "beacons_4744_rt"

# Connect to Beacon Data
setRedshiftEndpoint(dsn)
setTable(tableRt, tableType = "RESOURCE_TABLE")
#setTable(table)
;

***
## 1.2 Beacons Table & Schema
***

In [ ]:
include("../../Lib/Include-Package-v2.1.jl")
;

In [ ]:
# Create temp tables for Normal timeframe and Spike timeframe

urlNormal = UrlParamsInit()
urlSpike = UrlParamsInit()
urlCache = UrlParamsInit()
urlSummary = UrlParamsInit()

urlNormal.resourceTable = tableRt
urlNormal.rtView = "$(table)_rtnormal_view_prod"

urlSpike.resourceTable = tableRt
urlSpike.rtView = "$(table)_rtspike_view_prod"

timeNormal = anyTimeVar(2017,6,7,11,49,2017,6,7,11,59)
timeSpike  = anyTimeVar(2017,6,8,11,49,2017,6,8,11,59)

query("""create or replace view $(urlNormal.rtView) as (select * from $(urlNormal.resourceTable) where "timestamp" between $(timeNormal.startTimeMsUTC) and $(timeNormal.endTimeMsUTC))""")
query("""create or replace view $(urlSpike.rtView)  as (select * from $(urlSpike.resourceTable)  where "timestamp" between  $(timeSpike.startTimeMsUTC) and  $(timeSpike.endTimeMsUTC))""")


In [ ]:
query("""SELECT count(*) FROM $(urlNormal.rtView)""")

In [ ]:
query("""SELECT count(*) FROM $(urlSpike.rtView)""")

***
## Request Counters By URL Group
***

In [ ]:
requestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
requestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")

***
## Blocking Requests By URL Groups Across All Sessions
***

In [ ]:
blockingRequestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
blockingRequestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")
;

***
## Non Cache Requests Total By URL Groups Across All Sessions
***

In [ ]:
nonCacheRequestCountByGroupSDMRS(timeNormal,urlNormal,"Normal")
nonCacheRequestCountByGroupSDMRS(timeSpike,urlSpike,"Spike")
;

***
##Create the Cache Ratio
***

In [ ]:
cacheHitRatioSDMRS(timeNormal,urlNormal,"Normal")
cacheHitRatioSDMRS(timeSpike,urlSpike,"Spike")
;

***
## All Done
***

In [ ]:
q = query(""" drop view if exists $(urlNormal.rtView);""")
q = query(""" drop view if exists $(urlSpike.rtView);""")
;